In [ ]:
!pip3 install opencv-contrib-python

# Part 02: Image Retrieval

_You are suggested to work on this part after Lecture 06 (Multimedia Information Retrieval - Image Retrieval)_

In the second part of the assignment, you will work on a simple image retrieval problem, and implement parts of the Bag Of Words retrieval approach. You will work with images from the Mapillary Street-Level Sequences  (MSLS) data set. For an autonomous car, for instance, it is important to retrieve from a map or reference set of images the most similar images to what is being recording with a camera, in order to recognize a previously visited place.

You are asked to retrieve, from a collection of city images (we work with images from London), the most similar images to given queries. For this, we provide you a dataset, composed of:

* _database/_ folder: contains XXX images used as map (document collection)
* _query/_ folder: contains XXX images used as queries
* _database/database_lite.json_: list of images included in the map (the order is important because it correspond to the orded in the similarity matrix - see below)
* _query/query_lite.json_: list of images used for query (the order is important because it correspond to the orded in the similarity matrix - see below)
* _london_lite_gt.h5_: matrix of image similarity, containts all the information about relevant/non-relevant images 

__Relevance judgements (in the form of a matrix, the structure of which is explained below)__
For each query image, we provide a specification of which map images are relevant (1 in the london_lite_gt.h5 file) and which are not (0 in the london_lite_gt.h5). The content of the '.h5' file is a matrix, with rows corresponding to query images and columns corresponding to the map images. The matrix has thus has $N=500$ rows (where $N$ is the number of query images in the folder `query/images/`) and $M=1000$ columns (where $M$ is the number of query images in the folder `database/images/`). 

The $i$-th row of the matrix thus contains the relevance judgement for the $i$-th query image with respect to the map images. For instance, row 5 of the matrix is a vector of 1000 elements (one for each image in the map), in which each element is 1 if the corresponding map image is relevant for the query image 5, or $0$ in case it is not relevant.

Please note that in Python the index of a vector (or matrix) starts from 0 (instead of 1). This means that the first element of a vector has index $0$, and that, for example, the element with index $25$ corresponds to the 26-th element of the vector. 
Thus, as an example, the cell $(10, 25)$ contains the relevance of the 26-th map image for the $11$-th query image (i.e. is the 26-th image in the map relevan (1) or not (0) for the query image number 11?).
 

In the following, we provide you the code for loading the data set and to visualize the images.


> Note: for this assignment we use the '\_lite' version of the map and query set, which include a reduced number of images. This is to simplify the computations required to work on this assignment. For the final project, you can work using the full-sized sets of images from London. This will require longer computations, but you can obtain more reliable results.

## Load the query and map images
In the following we load the list of images in the map set (collection) and in the query set.
For each image, we have information also about the absolute coordinates of the camera that took the picture in the city of London. _Do not consider these coordinates for the exercises. We use them only to show the trajectory followed by the camera taking the pictures (see below)._ 

In [1]:
import json
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tqdm

# map
with open("data02/database/database_lite.json","r") as f:
    m_idx = json.load(f)
    m_imgs = np.array(m_idx["im_paths"])
    m_loc=np.array(m_idx["loc"])

# query
with open("data02/query/query_lite.json","r") as f:
    q_idx=json.load(f)
    q_imgs=np.array(q_idx["im_paths"])
    q_loc=np.array(q_idx["loc"])


In [2]:
with h5py.File("data02/london_lite_gt.h5","r") as f:
    fovs = f["fov"][:]
    sim = f["sim"][:].astype(np.uint8)

# ASSIGNMENT: Bag of Words representation
As seen in the lecture, in order to construct a dictionary of 'visual' terms (words) from a collection of images, one can extract local features from the images (e.g. keypoints) and use a clustering algorithm to automatically find groups (i.e. clusters) of similar local features. The centroid of a cluster is also a feature vector and can be considered as a term to be used for the search and retrieval of similar images.

In this assignment, we use the [$K$-Means clustering algorithm](https://www.youtube.com/watch?v=_aWzGGNrcic) (where $k$ is defined as a parameter by us) to extract the visual dictionary of terms from the images in the map collection (in the cells below you will find more information).

In the following code (please study it), we:
* load the list of images in the map collection
* for each image we extract a maximum of 50 ORB keypoints (local keypoint features). Each keypoint is described by a vector of 256 elements (keypoint descriptor) that describes the local characteristics of the image.
* use the set of keypoint descriptors (50 keypoints * 1000 images = 50000 vectors) as input of the k-Means clustering algorithm
* compute k=32 cluster centroids (which correspond to the 32 terms of our dictionary) - k is a configurable parameter

> __Note:__ Please, find more information about ORB in the [ORB paper](https://ieeexplore.ieee.org/document/6126544) and in the [Scikit Image ORB documentation](https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_orb.html).

> __Note 2:__ it may take some time to extract the descriptors from all images. Once extracted you can save them, and reload them when needed. If you change the number of centroids k, then you need to recompute the cluster centers.

### Generic CV feature extraction

In [3]:
import cv2 as cv
from tqdm import tqdm
import os

def generic_extractor(descriptor_extractor, imgs):
    descriptors = None

    for img_name in tqdm(imgs):
        img = cv.imread(os.path.join('data02/', img_name))
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
        _, descriptors_img = descriptor_extractor.detectAndCompute(img, None)
    
        if descriptors is None:
            descriptors = descriptors_img    
        else:
            descriptors = np.vstack((descriptors, descriptors_img))
    return descriptors

## Clustering
Following the Bag of Words framework (see the slides of the lecture), we can define the basic 'words' that compose the images via a clustering algorithm that takes the descriptors of the map images (training) as input. The basic 'words' form a so called visual vocabulary and are learned by grouping together those descriptor vectors that are most similar, i.e. their distance is lower than that between other descriptors.

We do it by using the $K$-Means algorithm: find [here](https://en.wikipedia.org/wiki/K-means_clustering) some more information about KMeans and [here](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) the reference documentation of the algorithm in Scikit-learn. Also an explanatory video with examples [here](https://www.youtube.com/watch?v=_aWzGGNrcic).

> Note: running the clustering may take few minutes. Once computed, you can save the centroids and subsequently load tham before use. If you change the value of K, you need to recompute the clusters.


In [4]:
import sklearn
from sklearn.cluster import KMeans

def train_cluster(descriptors):
    # clustering
    K = 32  # number of clusters (equivalent to the number of words) we want to estimate
    num_initialization = 5 # Number of time the k-means algorithm will be run with different centroid seeds.

    # Run the k-means clustering
    kmeans = KMeans(n_clusters=K, random_state=0, n_init=num_initialization, verbose=1)
    clusters = kmeans.fit(descriptors)  # we use the descriptors extracted from the map (training) images before
    centroids = clusters.cluster_centers_

    print("Shape of the centroids matrix: ", centroids.shape)
    print("We computed ", centroids.shape[0], "centroids of lengh ", centroids.shape[1], " (the same of the descriptor)")
    return kmeans, clusters, centroids
# Rememeber: the centroids can be considered as the words that compose our documents 
# -> in this case the basic components of the images


## Exercise 02.A: _'Bag of words' vector computation_

In order to compute the BoW representation of an image, we have to count the occurrences of the visual 'words', i.e. the local features, that we have learned in our visual vocabulary (using the $K$-Means clustering).

Given an image, we do the following:
   * we extract the local descriptors (we are using ORB descriptors in this assignment) 
   * for each descriptor we find the closest 'word' (i.e. the closes cluster centroid) in the visual vocabulary
   * we sum +1 to the counting of the occurence of the 'word' (in this way we construct the histogram of the occurrence of the visual 'words')
    
An image is thus described with a vector (of $K$ elements) that is an histogram of the occurrence of the words in the learned visual vocabulary (similar to the occurrence of words in a text document).  

In the following, implement the core of the function that computes the BoW vector. The function takes as input the centroids of the learned clusters (i.e. the vocabulary of the visual words) and the set of descriptors extracted from an image. It must return the BoW vector.

In [5]:
from tqdm import tqdm
import os
import skimage
from skimage.feature import ORB
from skimage.color import rgb2gray

# compute the bag of word vector for an image
def bag_of_words(kmeans, centroids, img_descriptors):
    n_centroids = centroids.shape[0]  # number of centroids found with the KMeans clustering
    n_descriptors = img_descriptors.shape[0]  # number of descriptors extracted from the image
    
    # initialization of the bag of words (BoW) vector
    # Note that the BoW vector has length equal to the number of cluster centroids
    # The cluster centroids are indeed our visual words, and the BoW will be the histogram of these words found in the given image
    bow_vector = np.zeros(n_centroids)  
    
    for i in range(n_descriptors):
        ## BEGIN ANSWER
        pred = kmeans.predict(img_descriptors[i].reshape(1, -1))
        bow_vector[pred[0]] += 1
        ## END ANSWER
    return bow_vector

### Bag of words representation of the map images 
Given a query image, our aim is to find similar images in our collection (i.e. map) images.
To do that, we have to convert the query image into its Bag of words (BoW) representation and look for similar vectors in the collection (i.e. map) images.

At this stage, since we will do many queries, it is handy to compute the BoW vectors for all images in the collection (i.e. map) and reuse them for multiple queries. (We convert all images in the collection to their BoW representation).

> It is advised to study and understand the code below (it may take some time to run).

In [6]:
def bow_map(descriptor_extractor, kmeans, centroids):
    bow_map_images = None
    # loop over the images in the map set
    for img_name in tqdm(m_imgs):
        # load image
        img = cv.imread(os.path.join('data02/', img_name))
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
        _, img_descriptors = descriptor_extractor.detectAndCompute(img, None)
    
        # compute BoW representation of the image (using the basic 'words', i.e. centroids, computed earlier)
        bow = bag_of_words(kmeans, centroids, img_descriptors)
        # add the computed BoW vector to the set of map representations
        if bow_map_images is None:
            bow_map_images = bow
        else:
            bow_map_images = np.vstack( (bow_map_images, bow))
    return bow_map_images

### Z-score normalization
In machine learning and data analysis problems, it is important to [normalize](https://en.wikipedia.org/wiki/Feature_scaling) (scale) the data, in order to avoid that features with larger range dominate the computation of distances (so skewing the computations). Here we use the Z-score normalization, meaning that we subtract the mean and divide by the variance (computed on the map images) of each feature (i.e. for each dimension of the BoW vectors). We use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) library of Scikit.

> __Note__: when computing the query BoW vectors, then, we need to also normalize them (using the mean and variance of the map vectors).

In [7]:
from sklearn import preprocessing
def normalise_images(bow_map_images):
    orig_bow_map_images = bow_map_images

    # Compute z-score statistics
    scaler = preprocessing.StandardScaler().fit(bow_map_images)
    # Normalize the vectors of the map collection (0 mean and 1 std)
    bow_map_images = scaler.transform(bow_map_images)
    return orig_bow_map_images, bow_map_images, scaler

## Exercise 02B: _Retrieve images_
We are ready to retrieve images from the collection (i.e. map) images, given the BoW representation of a query image. In order to do that, compute the Euclidean distance between the BoW vector of the query image and the BoW vector of all the collection images and return those with the smaller distance. 

_Hint:_ after computing the distances between the query image BoW vector and all BoW vectors of the map image, you may want to sort them in ascending order; the Numpy function [argsort](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html) may turn out to be useful.

> For this exercise you are asked to compute the Euclidean distance, but other distance metrics can also be used, e.g. Cosine distance, Minkowski Distance, Manhattan distance, etc. (this is left to you, if you want to explore them as well)

In [8]:
from scipy.spatial.distance import euclidean, cityblock, hamming, cosine
from scipy.spatial import minkowski_distance
# receives as input the:
#   - bag of words vectors of the map images
#   - the bag of work vector of the query image
def retrieve_images(map_bow_vectors, query_bow, distance_func):
    n_map_bow_vectors = map_bow_vectors.shape[0]
    bow_distances = np.zeros(n_map_bow_vectors)
    most_similar = None
    
    ## BEGIN ANSWER
    for i in range(n_map_bow_vectors):
        bow_distances[i] = distance_func(map_bow_vectors[i], query_bow)
    most_similar = np.argsort(bow_distances)
    ## END ANSWER
    
    return most_similar

## Exercise 02.C: _evaluation of performance_
As now you are able to retrieve images from the map that are similar (according to the BoW framework) to query images, you should evaluate the performance.

You can re-use the `precision_at_k()` function that you have developed for Assignment 01 (on text retrieval) and compute the precision@5 and precision@10 for the retrieval results of the previous exercise (`retrieved_images`).

In [9]:
## BEGIN ANSWER
def precision(relevant, retrieved):
    count = 0
    for d in retrieved:
        if d in relevant:
            count += 1
    if len(retrieved) == 0:
        return 0
    return count / len(retrieved)

def precision_at_k(relevant, retrieved, k):
    return precision(relevant, retrieved[:k])
## END ANSWER

## Time to evaluate the results!

In [13]:
def recall(relevant, retrieved):
    count = 0
    for d in retrieved:
        if d in relevant:
            count += 1
    if len(relevant) == 0:
        return 0
    return count / len(relevant)

def interpolated_precision_at_recall_X (relevant, retrieved, X):
    maxP = 0
    for rank in range(1, len(retrieved)+1):
        r = recall(relevant, retrieved[:rank])
        if r >= X:
            p = precision_at_k(relevant, retrieved, rank)
            maxP = max(maxP, p)
    return maxP

def average_precision(relevant, retrieved):
    total = 0
    for X in range(0, 11):
        total += interpolated_precision_at_recall_X(relevant, retrieved, X/10)
    return total/11

def mean_average_precision(all_relevant, all_retrieved):
    total = 0
    count = 0
    for qid in all_retrieved:
        total += average_precision(all_relevant[qid], all_retrieved[qid])
        count += 1
    return total / count

descriptor_extractors = {
    "ORB": cv.ORB_create(nfeatures = 100000),
    "BRISK": cv.BRISK_create()
}

distance_metrics = {
    "Hamming": hamming,
    "Euclidean": euclidean,
    "Manhattan": cityblock,
    "Cosine": cosine,
    "Minkowski": minkowski_distance
}

results = {}

best_MAP = 0
best_extractor = None
best_distance_metric = None

for descriptor_extractor in descriptor_extractors:
    results[descriptor_extractor] = {}
    
    print("Starting with", descriptor_extractor)
    descriptors = generic_extractor(descriptor_extractors[descriptor_extractor], m_imgs)
    kmeans, clusters, centroids = train_cluster(descriptors)
    bow_map_images = bow_map(descriptor_extractors[descriptor_extractor], kmeans, centroids)

    orig_bow_map_images, bow_map_images, scaler = normalise_images(bow_map_images)
    
    for distance_metric in distance_metrics:
        retrieved_images = {}
        relevant_images = {}
        print("Starting MAP measurement for", descriptor_extractor)
        for i in tqdm(range(len(q_imgs))):
            query_img_descriptors = generic_extractor(descriptor_extractors[descriptor_extractor], [q_imgs[i]])
            bow = bag_of_words(centroids, query_img_descriptors)

            # Normalize the query BoW vector using the mean and variance of the map (computed earlier and saved into the scaler object)
            bow = scaler.transform(bow.reshape(-1, 1).transpose())
            bow = bow.transpose().reshape(-1)

            # Retrieve the indices of the top-10 similar images from the map
            retrieved_images[i] = retrieve_images(bow_map_images, bow, cosine)
            # Indices of the relevant map images for the query: we have the relevance judgements (Ground truth)
            relevant_images[i] = np.where(sim[i, :] == 1)[0]
        
        MAP = mean_average_precision(relevant_images, retrieved_images)
        print(distance_metric, 'Mean Average Precision (MAP): %1.3f\n' % MAP)
        if MAP > best_MAP:
            best_MAP = MAP
            best_extractor = descriptor_extractor
            best_distance_metric = distance_metric

        results[descriptor_extractor][distance_metric] = MAP

print("Best P@10", best_P10)
print("Best extractor", best_extractor)
print("Best distance metric", best_distance_metric)
print(results)

Starting with KAZE


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:49<00:00,  9.11it/s]


Initialization complete
Iteration 0, inertia 127367.921875
Iteration 1, inertia 93402.71875
Iteration 2, inertia 91262.1171875
Iteration 3, inertia 90382.5859375
Iteration 4, inertia 89810.828125
Iteration 5, inertia 89352.125
Iteration 6, inertia 89078.5234375
Iteration 7, inertia 88884.1484375
Iteration 8, inertia 88731.890625
Iteration 9, inertia 88607.2421875
Iteration 10, inertia 88507.140625
Iteration 11, inertia 88430.7421875
Iteration 12, inertia 88371.703125
Iteration 13, inertia 88326.078125
Iteration 14, inertia 88291.7265625
Iteration 15, inertia 88265.9921875
Iteration 16, inertia 88247.09375
Iteration 17, inertia 88232.109375
Iteration 18, inertia 88219.3828125
Iteration 19, inertia 88208.484375
Iteration 20, inertia 88199.1640625
Iteration 21, inertia 88191.0859375
Iteration 22, inertia 88182.2578125
Iteration 23, inertia 88173.9375
Iteration 24, inertia 88165.96875
Iteration 25, inertia 88158.375
Iteration 26, inertia 88151.5390625
Iteration 27, inertia 88145.046875
Ite

Iteration 25, inertia 88515.6484375
Iteration 26, inertia 88505.65625
Iteration 27, inertia 88496.25
Iteration 28, inertia 88486.53125
Iteration 29, inertia 88477.0
Iteration 30, inertia 88466.96875
Iteration 31, inertia 88456.1171875
Iteration 32, inertia 88445.0703125
Iteration 33, inertia 88434.890625
Iteration 34, inertia 88425.6015625
Iteration 35, inertia 88416.6640625
Iteration 36, inertia 88408.9765625
Iteration 37, inertia 88403.0546875
Iteration 38, inertia 88398.6171875
Iteration 39, inertia 88394.265625
Iteration 40, inertia 88390.9609375
Iteration 41, inertia 88387.53125
Iteration 42, inertia 88384.890625
Iteration 43, inertia 88381.8125
Iteration 44, inertia 88378.953125
Iteration 45, inertia 88375.734375
Iteration 46, inertia 88373.09375
Iteration 47, inertia 88370.8515625
Iteration 48, inertia 88368.296875
Iteration 49, inertia 88366.0625
Iteration 50, inertia 88363.6484375
Iteration 51, inertia 88361.5625
Iteration 52, inertia 88358.6015625
Iteration 53, inertia 88355.

Iteration 7, inertia 88695.6484375
Iteration 8, inertia 88538.484375
Iteration 9, inertia 88416.5546875
Iteration 10, inertia 88326.640625
Iteration 11, inertia 88260.7109375
Iteration 12, inertia 88211.703125
Iteration 13, inertia 88173.828125
Iteration 14, inertia 88144.953125
Iteration 15, inertia 88122.8125
Iteration 16, inertia 88106.3203125
Iteration 17, inertia 88092.734375
Iteration 18, inertia 88081.03125
Iteration 19, inertia 88072.21875
Iteration 20, inertia 88065.046875
Iteration 21, inertia 88059.3828125
Iteration 22, inertia 88054.578125
Iteration 23, inertia 88050.7734375
Iteration 24, inertia 88047.515625
Iteration 25, inertia 88044.59375
Iteration 26, inertia 88042.46875
Iteration 27, inertia 88040.421875
Iteration 28, inertia 88038.015625
Iteration 29, inertia 88035.90625
Iteration 30, inertia 88034.09375
Iteration 31, inertia 88032.125
Iteration 32, inertia 88030.328125
Iteration 33, inertia 88028.84375
Iteration 34, inertia 88027.2421875
Iteration 35, inertia 88025.

Iteration 241, inertia 87870.7421875
Iteration 242, inertia 87870.609375
Iteration 243, inertia 87870.4375
Iteration 244, inertia 87870.3359375
Iteration 245, inertia 87870.375
Iteration 246, inertia 87870.4375
Iteration 247, inertia 87870.3203125
Iteration 248, inertia 87870.1875
Iteration 249, inertia 87870.2734375
Iteration 250, inertia 87870.265625
Iteration 251, inertia 87870.25
Iteration 252, inertia 87870.21875
Iteration 253, inertia 87870.1484375
Iteration 254, inertia 87870.140625
Iteration 255, inertia 87870.203125
Iteration 256, inertia 87870.171875
Iteration 257, inertia 87870.171875
Iteration 258, inertia 87870.1796875
Iteration 259, inertia 87870.1171875
Iteration 260, inertia 87870.2109375
Converged at iteration 260: center shift 4.531434001364687e-07 within tolerance 5.499655380845071e-07.
Initialization complete
Iteration 0, inertia 132467.265625
Iteration 1, inertia 94809.21875
Iteration 2, inertia 92088.75
Iteration 3, inertia 90866.3046875
Iteration 4, inertia 90128

Shape of the centroids matrix:  (32, 64)
We computed  32 centroids of lengh  64  (the same of the descriptor)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.24it/s]


Hamming P@10: 0.0
Euclidean P@10: 0.0
Manhattan P@10: 0.0
Cosine P@10: 0.0
Minkowski P@10: 0.0
Best P@10 0
Best extractor None
Best distance metric None


## Exercise 02.D (Bonus): _overall performance evaluation_
Extend the previous exercise by computing the Mean Average Precision (MAP) for the whole set of 500 query images.

__Note:__ Implementation of this exercise is completely on your own, without a given code structure, but try to reuse as much as possible the code already available (or that you developed in Assignment 01).


In [ ]:
## BEGIN ANSWER
# As cosine performed the best in the precision at k tests, we use this to calculate the MAP
def recall(relevant, retrieved):
    count = 0
    for d in retrieved:
        if d in relevant:
            count += 1
    if len(relevant) == 0:
        return 0
    return count / len(relevant)

def interpolated_precision_at_recall_X (relevant, retrieved, X):
    maxP = 0
    for rank in range(1, len(retrieved)+1):
        r = recall(relevant, retrieved[:rank])
        if r >= X:
            p = precision_at_k(relevant, retrieved, rank)
            maxP = max(maxP, p)
    return maxP

def average_precision(relevant, retrieved):
    total = 0
    for X in range(0, 11):
        total += interpolated_precision_at_recall_X(relevant, retrieved, X/10)
    return total/11

def mean_average_precision(all_relevant, all_retrieved):
    total = 0
    count = 0
    for qid in all_retrieved:
        total += average_precision(all_relevant[qid], all_retrieved[qid])
        count += 1
    return total / count

retrieved_images = {}
relevant_images = {}
for i in range(len(q_imgs)):
    img = plt.imread("data02/" + q_imgs[i])
    img = rgb2gray(img)
    # compute bag of words
    descriptor_extractor.detect_and_extract(img)  
    query_img_descriptors = descriptor_extractor.descriptors 
    bow = bag_of_words(centroids, query_img_descriptors)

    # Normalize the query BoW vector using the mean and variance of the map (computed earlier and saved into the scaler object)
    bow = scaler.transform(bow.reshape(-1, 1).transpose())
    bow = bow.transpose().reshape(-1)

    # Retrieve the indices of the top-10 similar images from the map
    retrieved_images[image] = retrieve_images(bow_map_images, bow, cosine)
    # Indices of the relevant map images for the query: we have the relevance judgements (Ground truth)
    relevant_images[image] = np.where(sim[i, :] == 1)[0]

print('Mean Average Precision (MAP): %1.3f\n' % mean_average_precision(relevant_images, retrieved_images))
## END ANSWER